In [18]:
import geopandas as gpd
import numpy as np
from shapely.geometry import Point
from tqdm import tqdm

# Load UK shapefile (EPSG:27700 - British National Grid)
uk = gpd.read_file("data/CTRY_DEC_2024_UK_BFC.shp")
uk = uk[uk['CTRY24NM'].isin(['England', 'Scotland', 'Wales', 'Northern Ireland'])]

# Merge into a single MultiPolygon (for clean join)
uk_union = uk.geometry.unary_union
uk_gdf = gpd.GeoDataFrame(geometry=[uk_union], crs='EPSG:27700')

# Get bounds
minx, miny, maxx, maxy = uk_union.bounds

# Grid spacing in meters
spacing = 1000

# Create grid of points
x_coords = np.arange(minx, maxx, spacing)
y_coords = np.arange(miny, maxy, spacing)

# Generate all combinations using numpy meshgrid
xx, yy = np.meshgrid(x_coords, y_coords)
xx_flat = xx.ravel()
yy_flat = yy.ravel()

# Progress bar while building points
print("Generating points...")
points = [Point(x, y) for x, y in tqdm(zip(xx_flat, yy_flat), total=len(xx_flat))]

# Convert to GeoDataFrame
grid_gdf = gpd.GeoDataFrame(geometry=points, crs='EPSG:27700')

# Spatial join to keep only points within UK
print("Filtering points inside UK...")
points_within_uk = gpd.sjoin(grid_gdf, uk_gdf, predicate='within', how='inner')

# Summary
print(f"\nTotal grid points: {len(grid_gdf):,}")
print(f"Points inside UK: {len(points_within_uk):,}")


/var/folders/0n/sggn66691kd6g5mtyzpq2hv80000gn/T/ipykernel_54338/1213105582.py:11: DeprecationWarning: The 'unary_union' attribute is deprecated, use the 'union_all()' method instead.
  uk_union = uk.geometry.unary_union


Generating points...


100%|██████████| 797040/797040 [00:04<00:00, 182546.54it/s]


Filtering points inside UK...

Total grid points: 797,040
Points inside UK: 244,359


In [19]:
# Reproject to WGS84 (EPSG:4326) if you want lat/lon in the CSV
points_latlon = points_within_uk.to_crs('EPSG:4326')

# Extract lat/lon from geometry
points_latlon['longitude'] = points_latlon.geometry.x
points_latlon['latitude'] = points_latlon.geometry.y

# Save selected columns to CSV
points_latlon[['longitude', 'latitude']].to_csv("uk_grid_1000m.csv", index=False)

print("Saved to 'uk_grid_1000m.csv'")


Saved to 'uk_grid_1000m.csv'
